<a href="https://colab.research.google.com/github/nattntn/OPG-SHAPer_Experiment/blob/main/OPG_SHAPer_95percentile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SHAP

## Import data set (เรียกใช้ CSV)

In [ ]:
import pandas as pd
import shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import io

In [ ]:
#เชื่อม google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## เลือกข้อมูล ทำ Background data ของ data train

In [ ]:
dt_train = pd.read_csv (r'/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model/Multi_task(Age)(7-25)/Table_train_Predict.csv')
dt_train

,Unnamed: 0,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Fold,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V1.jpg,2,0,Female,7.107492,7.0,0.000003,Female,0.0
1,1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV1.jpg,4,0,Female,7.020138,7.0,0.000025,Female,0.0
2,2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V2.jpg,10,0,Female,6.975561,7.0,0.000808,Female,0.0
3,3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV2.jpg,9,0,Female,6.938187,7.0,0.000107,Female,0.0
4,4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V3.jpg,10,0,Female,6.916692,7.0,0.000014,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184,4184,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J375.jpg,3,1,Male,25.299234,25.0,0.999988,Male,1.0
4185,4185,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J437.jpg,5,1,Male,24.729193,25.0,0.999987,Male,1.0
4186,4186,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J480.jpg,5,1,Male,24.564425,25.0,1.000000,Male,1.0
4187,4187,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J491.jpg,6,1,Male,24.113144,24.0,1.000000,Male,1.0


In [ ]:
df = dt_train.drop(dt_train.columns[0], axis=1)
df

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Fold,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V1.jpg,2,0,Female,7.107492,7.0,0.000003,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV1.jpg,4,0,Female,7.020138,7.0,0.000025,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V2.jpg,10,0,Female,6.975561,7.0,0.000808,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV2.jpg,9,0,Female,6.938187,7.0,0.000107,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V3.jpg,10,0,Female,6.916692,7.0,0.000014,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J375.jpg,3,1,Male,25.299234,25.0,0.999988,Male,1.0
4185,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J437.jpg,5,1,Male,24.729193,25.0,0.999987,Male,1.0
4186,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J480.jpg,5,1,Male,24.564425,25.0,1.000000,Male,1.0
4187,25,M,Normal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J491.jpg,6,1,Male,24.113144,24.0,1.000000,Male,1.0


In [ ]:
# นับรูป
# Get unique age values
unique_ages = dt_train['Age(year)'].unique()

# Iterate through unique ages and calculate shape for 'M' gender
for age in unique_ages:
    shape = dt_train[(dt_train['Age(year)'] == age) & (dt_train['Gender'] == 'M')].shape
    print(f"Age {age}: {shape}")

# Get unique age values
unique_ages = dt_train['Age(year)'].unique()

# Iterate through unique ages and calculate shape for 'F' gender
for age in unique_ages:
    shape = dt_train[(dt_train['Age(year)'] == age) & (dt_train['Gender'] == 'F')].shape
    print(f"Age {age}: {shape}")

Age 7: (111, 15)
Age 8: (116, 15)
Age 9: (122, 15)
Age 10: (114, 15)
Age 11: (114, 15)
Age 12: (119, 15)
Age 13: (106, 15)
Age 14: (110, 15)
Age 15: (110, 15)
Age 16: (112, 15)
Age 17: (107, 15)
Age 18: (104, 15)
Age 19: (107, 15)
Age 20: (105, 15)
Age 21: (107, 15)
Age 22: (97, 15)
Age 23: (108, 15)
Age 24: (104, 15)
Age 25: (96, 15)
Age 7: (114, 15)
Age 8: (116, 15)
Age 9: (120, 15)
Age 10: (116, 15)
Age 11: (117, 15)
Age 12: (117, 15)
Age 13: (108, 15)
Age 14: (113, 15)
Age 15: (117, 15)
Age 16: (118, 15)
Age 17: (112, 15)
Age 18: (128, 15)
Age 19: (115, 15)
Age 20: (99, 15)
Age 21: (100, 15)
Age 22: (105, 15)
Age 23: (101, 15)
Age 24: (91, 15)
Age 25: (113, 15)


In [ ]:
# เลือกรูปจาก train มาทำ bg data
import pandas as pd
import random

# Sample DataFrame (replace this with your actual DataFrame)
data = {
    'Age(year)': dt_train['Age(year)'],
    'Gender': dt_train['Gender'],
    'Folder_type': dt_train['Folder_type'],
    'Folder_name': dt_train['Folder_name'],
    'Path_Name': dt_train['Path_Name'],
    'Filename': dt_train['Filename'],
}

df = pd.DataFrame(data)

# Calculate the desired number of rows per unique age
desired_per_age = 2660 // (25 - 7 + 1)

# Initialize a list to store selected rows
selected_rows = []

# Iterate through unique age values
for age in range(7, 26):
    # Get all rows with the current age
    age_rows = df[df['Age(year)'] == age]

    # Randomly shuffle the rows for this age
    age_rows = age_rows.sample(frac=1, random_state=42)

    # Determine the number of rows to select for each gender
    rows_per_gender = desired_per_age // 2

    # Select rows for both "Male" and "Female" genders
    selected_male_rows = age_rows[age_rows['Gender'] == 'M'].head(rows_per_gender)
    selected_female_rows = age_rows[age_rows['Gender'] == 'F'].head(rows_per_gender)

    # Append the selected rows to the list
    selected_rows.extend([selected_male_rows, selected_female_rows])

# Concatenate the selected rows for each age into a final DataFrame
selected_dft = pd.concat(selected_rows)

# Now, selected_df contains 950 rows with an approximately even distribution of ages from 7 to 25
# and an equal number of "Male" and "Female" samples for each age.


In [ ]:
selected_dft

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename
184,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,VV116.jpg
120,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,VV13.jpg
207,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipVV244.jpg
148,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,VV50.jpg
214,7,M,Normal,Lt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipVV28.jpg
...,...,...,...,...,...,...
4050,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J245.jpg
4044,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J218.jpg
4024,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J148.jpg
4020,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J144.jpg


In [ ]:
#selected_dft.to_csv('/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model/Multi_task(Age)(7-25)/Bg_train.csv')  # Setting index=False to avoid saving the row numbers as a column

In [ ]:
# Get unique age values
unique_ages = selected_dft['Age(year)'].unique()

# Iterate through unique ages and calculate shape for 'M' gender
for age in unique_ages:
    shape = selected_dft[(selected_dft['Age(year)'] == age) & (selected_dft['Gender'] == 'M')].shape
    print(f"Age {age}: {shape}")

# Get unique age values
unique_ages = selected_dft['Age(year)'].unique()

# Iterate through unique ages and calculate shape for 'F' gender
for age in unique_ages:
    shape = selected_dft[(selected_dft['Age(year)'] == age) & (selected_dft['Gender'] == 'F')].shape
    print(f"Age {age}: {shape}")

Age 7: (70, 6)
Age 8: (70, 6)
Age 9: (70, 6)
Age 10: (70, 6)
Age 11: (70, 6)
Age 12: (70, 6)
Age 13: (70, 6)
Age 14: (70, 6)
Age 15: (70, 6)
Age 16: (70, 6)
Age 17: (70, 6)
Age 18: (70, 6)
Age 19: (70, 6)
Age 20: (70, 6)
Age 21: (70, 6)
Age 22: (70, 6)
Age 23: (70, 6)
Age 24: (70, 6)
Age 25: (70, 6)
Age 7: (70, 6)
Age 8: (70, 6)
Age 9: (70, 6)
Age 10: (70, 6)
Age 11: (70, 6)
Age 12: (70, 6)
Age 13: (70, 6)
Age 14: (70, 6)
Age 15: (70, 6)
Age 16: (70, 6)
Age 17: (70, 6)
Age 18: (70, 6)
Age 19: (70, 6)
Age 20: (70, 6)
Age 21: (70, 6)
Age 22: (70, 6)
Age 23: (70, 6)
Age 24: (70, 6)
Age 25: (70, 6)


## เลือกข้อมูล ทำ Background data ของ data test

In [ ]:
df_test_age = pd.read_csv (r'/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model/Multi_task(Age)(7-25)/Table_test_Predict.csv')
df_test_age = df_test_age.iloc[:,1:]
print(df_test_age.shape)
df_test_age

(943, 13)


,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V69.jpg,0,Female,7.423399,7.0,5.141945e-03,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV69.jpg,0,Female,7.415316,7.0,4.618836e-02,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V70.jpg,0,Female,7.548301,8.0,6.406498e-03,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV70.jpg,0,Female,7.492433,7.0,2.021052e-05,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V74.jpg,0,Female,7.343049,7.0,2.091009e-02,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J132.jpg,0,Female,21.686228,22.0,1.800820e-05,Female,0.0
939,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J145.jpg,0,Female,21.118502,21.0,1.193671e-01,Female,0.0
940,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J149.jpg,0,Female,19.639433,20.0,1.212668e-03,Female,0.0
941,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J158.jpg,0,Female,21.359797,21.0,3.044503e-09,Female,0.0


In [ ]:
df_test_age['Age_predict_int'] = df_test_age['Age_predict_int'].astype(int)
df_test_age

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V69.jpg,0,Female,7.423399,7,5.141945e-03,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV69.jpg,0,Female,7.415316,7,4.618836e-02,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V70.jpg,0,Female,7.548301,8,6.406498e-03,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV70.jpg,0,Female,7.492433,7,2.021052e-05,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V74.jpg,0,Female,7.343049,7,2.091009e-02,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J132.jpg,0,Female,21.686228,22,1.800820e-05,Female,0.0
939,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J145.jpg,0,Female,21.118502,21,1.193671e-01,Female,0.0
940,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J149.jpg,0,Female,19.639433,20,1.212668e-03,Female,0.0
941,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J158.jpg,0,Female,21.359797,21,3.044503e-09,Female,0.0


In [ ]:
# # เลือกที่อายุตรงกัน
# e = df_test_age[df_test_age['Age(year)'] == df_test_age['Age_predict_int']]
# e

In [ ]:
df_test_age = df_test_age.sort_values(by='Age(year)', kind='mergesort')
df_test_age

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V69.jpg,0,Female,7.423399,7,5.141945e-03,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV69.jpg,0,Female,7.415316,7,4.618836e-02,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V70.jpg,0,Female,7.548301,8,6.406498e-03,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV70.jpg,0,Female,7.492433,7,2.021052e-05,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V74.jpg,0,Female,7.343049,7,2.091009e-02,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J132.jpg,0,Female,21.686228,22,1.800820e-05,Female,0.0
939,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J145.jpg,0,Female,21.118502,21,1.193671e-01,Female,0.0
940,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J149.jpg,0,Female,19.639433,20,1.212668e-03,Female,0.0
941,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J158.jpg,0,Female,21.359797,21,3.044503e-09,Female,0.0


In [ ]:
df_test_age['Age(year)'].unique()

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25])

In [ ]:
selected_test = df_test_age
selected_test

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V69.jpg,0,Female,7.423399,7,5.141945e-03,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV69.jpg,0,Female,7.415316,7,4.618836e-02,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V70.jpg,0,Female,7.548301,8,6.406498e-03,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV70.jpg,0,Female,7.492433,7,2.021052e-05,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V74.jpg,0,Female,7.343049,7,2.091009e-02,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J132.jpg,0,Female,21.686228,22,1.800820e-05,Female,0.0
939,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J145.jpg,0,Female,21.118502,21,1.193671e-01,Female,0.0
940,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J149.jpg,0,Female,19.639433,20,1.212668e-03,Female,0.0
941,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J158.jpg,0,Female,21.359797,21,3.044503e-09,Female,0.0


## ดูข้อมูล

#### age

In [ ]:
test_age = e

NameError: name 'e' is not defined

In [ ]:
test_age

In [ ]:
# check ว่าในเเต่ละช่วงอายุมีกี่คน
# Get unique age values
unique_ages = test_age['Age(year)'].unique()

# Iterate through unique ages and calculate shape for 'M' gender
for age in unique_ages:
    shape = test_age[(test_age['Age(year)'] == age)].shape
    print(f"Age {age}: {shape}")

In [ ]:
# Group by 'Age(year)' and 'Gender' and calculate the count for each group
age_gender_counts = test_age.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# Print the results
for index, row in age_gender_counts.iterrows():
    print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")


In [ ]:
# test_age.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_age_all.csv')

In [ ]:
# Group by 'Age(year)' and 'Gender' and calculate the count for each group
age_gender_counts = test_age.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
age_counts = test_age.groupby('Age(year)').size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
gender_counts = test_age.groupby('Gender').size().reset_index(name='Count')


# Print the breakdown by age and gender
print("Breakdown by age and gender:")
for index, row in age_gender_counts.iterrows():
    print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")

# Print the summary per age
print("\nSummary per age:")
for index, row in age_counts.iterrows():
    print(f"Age {row['Age(year)']}: {row['Count']}")

# Print the summary per gender
print("\nSummary per gender:")
for index, row in gender_counts.iterrows():
    print(f"Gender {row['Gender']}: {row['Count']}")

#### gender

In [ ]:
df_test_age

In [ ]:
# เลือกเฉพาะ row ที่ทายถูก
c = df_test_age[((df_test_age['Gender_predict_str'] == 'Female') & (df_test_age['Gender'] == 'F')) | ((df_test_age['Gender_predict_str'] == 'Male') & (df_test_age['Gender'] == 'M'))]
c

In [ ]:
# ทำค่า Prob
c['prob'] = c.apply(lambda row: 1 - row['Gender_predict'] if row['Gender'] == 'F' else row['Gender_predict'], axis=1)

In [ ]:
c['Age(year)'].unique()

In [ ]:
# Select rows where 'prob' is greater than 0.9
d = c[c['prob'] > 0.90]

# Display the selected rows
d

In [ ]:
# check ว่าในเเต่ละช่วงอายุมีกี่คน
# Get unique age values
unique_ages = d['Age(year)'].unique()

# Iterate through unique ages and calculate shape for 'M' gender
for age in unique_ages:
    shape = d[(d['Age(year)'] == age)].shape
    print(f"Age {age}: {shape}")

In [ ]:
test_gender = d
test_gender

In [ ]:
# test_gender.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_gender_Prob_morethan_90.csv')

In [ ]:
# Group by 'Age(year)' and 'Gender' and calculate the count for each group
age_gender_counts = test_gender.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
age_counts = test_gender.groupby('Age(year)').size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
gender_counts = test_gender.groupby('Gender').size().reset_index(name='Count')


# Print the breakdown by age and gender
print("Breakdown by age and gender:")
for index, row in age_gender_counts.iterrows():
    print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")

# Print the summary per age
print("\nSummary per age:")
for index, row in age_counts.iterrows():
    print(f"Age {row['Age(year)']}: {row['Count']}")

# Print the summary per gender
print("\nSummary per gender:")
for index, row in gender_counts.iterrows():
    print(f"Gender {row['Gender']}: {row['Count']}")

#### ดูที่ซ้ำ

include only the rows where the Filename exists in the test_age DataFrame, you can use the isin() function.

##### age

In [ ]:
# test_age_duplicated_gender = test_age[test_age['Filename'].isin(test_gender['Filename'])] # test_age แสดงเฉพาะ แถวที่มี Filename ซ้ำกับ test_gender
# test_age_duplicated_gender

In [ ]:
# # Group by 'Age(year)' and 'Gender' and calculate the count for each group
# age_gender_counts = test_age_duplicated_gender.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# # Group by 'Age(year)' and calculate the count for each age
# age_counts = test_age_duplicated_gender.groupby('Age(year)').size().reset_index(name='Count')

# # Group by 'Age(year)' and calculate the count for each age
# gender_counts = test_age_duplicated_gender.groupby('Gender').size().reset_index(name='Count')


# # Print the breakdown by age and gender
# print("Breakdown by age and gender:")
# for index, row in age_gender_counts.iterrows():
#     print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")

# # Print the summary per age
# print("\nSummary per age:")
# for index, row in age_counts.iterrows():
#     print(f"Age {row['Age(year)']}: {row['Count']}")

# # Print the summary per gender
# print("\nSummary per gender:")
# for index, row in gender_counts.iterrows():
#     print(f"Gender {row['Gender']}: {row['Count']}")

In [ ]:
# import os
# os.makedirs("/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data")

In [ ]:
# test_age_duplicated_gender.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_age_duplicated_gender.csv')

In [ ]:
test_age_only =  test_age[~test_age['Filename'].isin(test_gender['Filename'])] # test_age ไม่แสดง แถวที่มี Filename ซ้ำกับ test_gender
test_age_only

In [ ]:
# Group by 'Age(year)' and 'Gender' and calculate the count for each group
age_gender_counts = test_age_only.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
age_counts = test_age_only.groupby('Age(year)').size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
gender_counts = test_age_only.groupby('Gender').size().reset_index(name='Count')


# Print the breakdown by age and gender
print("Breakdown by age and gender:")
for index, row in age_gender_counts.iterrows():
    print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")

# Print the summary per age
print("\nSummary per age:")
for index, row in age_counts.iterrows():
    print(f"Age {row['Age(year)']}: {row['Count']}")

# Print the summary per gender
print("\nSummary per gender:")
for index, row in gender_counts.iterrows():
    print(f"Gender {row['Gender']}: {row['Count']}")

In [ ]:
# test_age_only.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_age_only.csv')

##### gender

In [ ]:
test_age_duplicated_gender = test_gender[test_gender['Filename'].isin(test_age['Filename'])]
test_age_duplicated_gender

In [ ]:
# Group by 'Age(year)' and 'Gender' and calculate the count for each group
age_gender_counts = test_age_duplicated_gender.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
age_counts = test_age_duplicated_gender.groupby('Age(year)').size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
gender_counts = test_age_duplicated_gender.groupby('Gender').size().reset_index(name='Count')


# Print the breakdown by age and gender
print("Breakdown by age and gender:")
for index, row in age_gender_counts.iterrows():
    print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")

# Print the summary per age
print("\nSummary per age:")
for index, row in age_counts.iterrows():
    print(f"Age {row['Age(year)']}: {row['Count']}")

# Print the summary per gender
print("\nSummary per gender:")
for index, row in gender_counts.iterrows():
    print(f"Gender {row['Gender']}: {row['Count']}")


In [ ]:
# test_age_duplicated_gender.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_age_duplicated_gender.csv')

In [ ]:
test_gender_only = test_gender[~test_gender['Filename'].isin(test_age['Filename'])]
test_gender_only

In [ ]:
# Group by 'Age(year)' and 'Gender' and calculate the count for each group
age_gender_counts = test_gender_only.groupby(['Age(year)', 'Gender']).size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
age_counts = test_gender_only.groupby('Age(year)').size().reset_index(name='Count')

# Group by 'Age(year)' and calculate the count for each age
gender_counts = test_gender_only.groupby('Gender').size().reset_index(name='Count')


# Print the breakdown by age and gender
print("Breakdown by age and gender:")
for index, row in age_gender_counts.iterrows():
    print(f"Age {row['Age(year)']}, Gender {row['Gender']}: {row['Count']}")

# Print the summary per age
print("\nSummary per age:")
for index, row in age_counts.iterrows():
    print(f"Age {row['Age(year)']}: {row['Count']}")

# Print the summary per gender
print("\nSummary per gender:")
for index, row in gender_counts.iterrows():
    print(f"Gender {row['Gender']}: {row['Count']}")


In [ ]:
# test_gender_only.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_gender_only.csv')

## Create background

In [ ]:
def process_input(dt_train):
    # Initialize an empty list to store background data
    background_data = []

    # Ensure 'Path_Name' column exists in the DataFrame
    if 'Path_Name' not in dt_train.columns:
        raise ValueError("The 'Path_Name' column does not exist in the DataFrame.")

    # Loop to preprocess and create the background data
    for i in range(len(dt_train)):
        print(f"Processing image {i+1}/{len(dt_train)}")
        # Get the image path from the DataFrame
        image_path = dt_train['Path_Name'].iloc[i]  # Use .iloc to access rows by integer index
        # Load and preprocess the image
        image = load_img(image_path, target_size=(224, 224))  # Adjust target_size
        preprocessed_image = img_to_array(image) / 255.0  # Normalize pixel values

        # Add the preprocessed image to the background_data list
        background_data.append(preprocessed_image)

    return background_data

### data train

In [ ]:
import pandas as pd
import numpy as np
import torch
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
sdf_train = selected_dft.sort_values(by='Age(year)', kind='mergesort')
sdf_train

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename
184,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,VV116.jpg
120,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,VV13.jpg
207,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipVV244.jpg
148,7,M,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,VV50.jpg
214,7,M,Normal,Lt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipVV28.jpg
...,...,...,...,...,...,...
4050,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J245.jpg
4044,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J218.jpg
4024,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J148.jpg
4020,25,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J144.jpg


In [ ]:
sdf_train['Age(year)'].unique()

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25])

In [ ]:
background_train = process_input(sdf_train)

Processing image 1/2660
Processing image 2/2660
Processing image 3/2660
Processing image 4/2660
Processing image 5/2660
Processing image 6/2660
Processing image 7/2660
Processing image 8/2660
Processing image 9/2660
Processing image 10/2660
Processing image 11/2660
Processing image 12/2660
Processing image 13/2660
Processing image 14/2660
Processing image 15/2660
Processing image 16/2660
Processing image 17/2660
Processing image 18/2660
Processing image 19/2660
Processing image 20/2660
Processing image 21/2660
Processing image 22/2660
Processing image 23/2660
Processing image 24/2660
Processing image 25/2660
Processing image 26/2660
Processing image 27/2660
Processing image 28/2660
Processing image 29/2660
Processing image 30/2660
Processing image 31/2660
Processing image 32/2660
Processing image 33/2660
Processing image 34/2660
Processing image 35/2660
Processing image 36/2660
Processing image 37/2660
Processing image 38/2660
Processing image 39/2660
Processing image 40/2660
Processin

### data test (943 img)

In [ ]:
print(df_test_age.shape)
df_test_age.head()

(943, 13)


,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V69.jpg,0,Female,7.423399,7,0.005142,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV69.jpg,0,Female,7.415316,7,0.046188,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V70.jpg,0,Female,7.548301,8,0.006406,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV70.jpg,0,Female,7.492433,7,0.000020,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V74.jpg,0,Female,7.343049,7,0.020910,Female,0.0


In [ ]:
sdf_test = df_test_age.sort_values(by='Age(year)', kind='mergesort')
sdf_test

,Age(year),Gender,Folder_type,Folder_name,Path_Name,Filename,Gender_Class,Gender_Predict,Age_predict,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V69.jpg,0,Female,7.423399,7,5.141945e-03,Female,0.0
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV69.jpg,0,Female,7.415316,7,4.618836e-02,Female,0.0
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V70.jpg,0,Female,7.548301,8,6.406498e-03,Female,0.0
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,FlipV70.jpg,0,Female,7.492433,7,2.021052e-05,Female,0.0
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,V74.jpg,0,Female,7.343049,7,2.091009e-02,Female,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J132.jpg,0,Female,21.686228,22,1.800820e-05,Female,0.0
939,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J145.jpg,0,Female,21.118502,21,1.193671e-01,Female,0.0
940,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J149.jpg,0,Female,19.639433,20,1.212668e-03,Female,0.0
941,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,J158.jpg,0,Female,21.359797,21,3.044503e-09,Female,0.0


In [ ]:
sdf_test = sdf_test.reset_index().iloc[:,1:]

In [ ]:
sdf_test['Age(year)'].unique()

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25])

In [ ]:
background_test = process_input(sdf_test)

Processing image 1/943
Processing image 2/943
Processing image 3/943
Processing image 4/943
Processing image 5/943
Processing image 6/943
Processing image 7/943
Processing image 8/943
Processing image 9/943
Processing image 10/943
Processing image 11/943
Processing image 12/943
Processing image 13/943
Processing image 14/943
Processing image 15/943
Processing image 16/943
Processing image 17/943
Processing image 18/943
Processing image 19/943
Processing image 20/943
Processing image 21/943
Processing image 22/943
Processing image 23/943
Processing image 24/943
Processing image 25/943
Processing image 26/943
Processing image 27/943
Processing image 28/943
Processing image 29/943
Processing image 30/943
Processing image 31/943
Processing image 32/943
Processing image 33/943
Processing image 34/943
Processing image 35/943
Processing image 36/943
Processing image 37/943
Processing image 38/943
Processing image 39/943
Processing image 40/943
Processing image 41/943
Processing image 42/943
P

In [ ]:
print(type(background_test))

<class 'list'>


##**แปลง Bg dataเป็น array เพื่อใช้ในการคำนวณค่า SHAP**

In [ ]:
background_train_array = np.array(background_train)
background_test_array = np.array(background_test)

In [ ]:
print(len(background_train_array))
print(background_train_array.shape)
print(background_test_array.shape)

2660
(2660, 224, 224, 3)
(943, 224, 224, 3)


# คำนวณ SHAP Values

## load model

### Import

In [ ]:
import os
import numpy as np
import pandas as pd
#import efficientnet.tfkeras
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0

### กำหนดค่าพารามิเตอร์ (ตามที่กำหนดใน Model)


In [ ]:
batch_size = 16
width = 224
height = 224
epochs = 250 # รอบในการเทรน
dropout_rate = 0.2
input_shape = (height, width, 3)

### Clone Efficient

In [ ]:
#ดึงข้อมูลใน Github มาใช้
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/


/content
Cloning into 'efficientnet_keras_transfer_learning'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1100 (delta 133), reused 241 (delta 121), pack-reused 837
Receiving objects: 100% (1100/1100), 14.10 MiB | 16.00 MiB/s, done.
Resolving deltas: 100% (630/630), done.
/content/efficientnet_keras_transfer_learning


### Load Model

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model/Multi_task(Age)(7-25)/16_Multi_8e-6_250_Unfreeze.h5')

from efficientnet.layers import Swish, DropConnect
from efficientnet.model import ConvKernalInitializer
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({
    'ConvKernalInitializer': ConvKernalInitializer,
    'Swish': Swish,
    'DropConnect':DropConnect
})


In [ ]:
from tensorflow.keras.models import load_model
model = tf.keras.models.load_model("/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model/Multi_task(Age)(7-25)/16_Multi_8e-6_250_Unfreeze.h5")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 swish (Swish)               (None, 112, 112, 32)         0         ['batch_normalization[0][0

In [ ]:
height = width = model.input_shape[1]
print(f"height: {height}, width: {width}")

height: 224, width: 224


In [ ]:
# Check the number of output units in the final layer
num_output_units = model.layers[-1].output_shape[-1]

# Check if it's a binary classification model (2 output units) or not
if num_output_units == 2:
    print("This is a binary classification model.")
else:
    print("This is not a binary classification model.")

This is not a binary classification model.


## SHAP & YOLO

### SHAP

In [ ]:
!pip install shap==0.44.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
import shap

In [ ]:
print(background_test_array.shape)
print(background_test_array[[10]].shape) # ภาพที่ 10

(943, 224, 224, 3)
(1, 224, 224, 3)


In [ ]:
model.output_names

['prediction_layer', 'prediction_layer2']

In [ ]:
model.output_names[0] # age estimation

'prediction_layer'

In [ ]:
model.output_names[1] # gender prediction

'prediction_layer2'

เลือกเฉพาะ Output layer ไปทำการหาความสำคัญ

In [ ]:
import shap
import tensorflow as tf

# Create separate models for each output you want to explain
#AGE
model_layer1 = tf.keras.Model(inputs=model.input, outputs=model.get_layer('prediction_layer').output)
#Gender
model_layer2 = tf.keras.Model(inputs=model.input, outputs=model.get_layer('prediction_layer2').output)


In [ ]:
# Define your explainer for the first output layer (prediction_layer)
#AGE
explainer_layer1 = shap.GradientExplainer(model_layer1, background_train_array)

In [ ]:
# Define your explainer for the second output layer (prediction_layer2)
#Gender
explainer_layer2 = shap.GradientExplainer(model_layer2, background_train_array)

### yolo

In [ ]:
# load YOLO ที่Train มา
# Replace with code to load your trained YOLO model
weights_path = '/content/gdrive/MyDrive/Tooth_Shap_GPT/papers/gpt/Yolo_v5/best.pt'

# Load YOLOv5 model from Google Drive
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)

You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 141.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 145.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 153.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 8.3s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-6-12 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


#### ทํา non maximum suppression

In [ ]:
# import pandas as pd

# def compute_iou(boxA, boxB):
#     # Determine the coordinates of the intersection rectangle
#     xA = max(boxA[0], boxB[0])
#     yA = max(boxA[1], boxB[1])
#     xB = min(boxA[2], boxB[2])
#     yB = min(boxA[3], boxB[3])

#     # Compute the area of intersection
#     interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

#     # Compute the area of both the prediction and ground-truth rectangles
#     boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
#     boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

#     # Compute the intersection over union by taking the intersection
#     # area and dividing it by the sum of prediction + ground-truth
#     # areas - the interesection area
#     iou = interArea / float(boxAArea + boxBArea - interArea)

#     return iou

# def nms_per_class(df, iou_threshold=0.5):
#     # Initialize an empty DataFrame to store NMS results
#     df_nms = pd.DataFrame()

#     # Get unique class IDs
#     class_ids = df['class'].unique()

#     for class_id in class_ids:
#         # Filter detections by class
#         df_class = df[df['class'] == class_id].copy()

#         # Apply NMS
#         df_class_sorted = df_class.sort_values(by='confidence', ascending=False).reset_index(drop=True)
#         suppressed_indices = set()

#         for i in range(len(df_class_sorted)):
#             if i in suppressed_indices:
#                 continue

#             for j in range(i+1, len(df_class_sorted)):
#                 if j in suppressed_indices:
#                     continue

#                 boxA = df_class_sorted.iloc[i][['xmin', 'ymin', 'xmax', 'ymax']]
#                 boxB = df_class_sorted.iloc[j][['xmin', 'ymin', 'xmax', 'ymax']]

#                 if compute_iou(boxA, boxB) > iou_threshold:
#                     suppressed_indices.add(j)

#         # Filter out suppressed detections
#         df_nms_class = df_class_sorted.drop(index=list(suppressed_indices)).reset_index(drop=True)

#         # Append results for this class to the main DataFrame
#         df_nms = pd.concat([df_nms, df_nms_class], ignore_index=True)

#     return df_nms

In [ ]:
# กำหนด confidence
import pandas as pd

def compute_iou(boxA, boxB):
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxB[3], boxB[3])

    # Compute the area of intersection
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

def nms_per_class(df, iou_threshold=0.5, confidence_threshold=0.6):
    # Initialize an empty DataFrame to store NMS results
    df_nms = pd.DataFrame()

    # Get unique class IDs
    class_ids = df['class'].unique()

    for class_id in class_ids:
        # Filter detections by class
        df_class = df[df['class'] == class_id].copy()

        # Filter out detections with low confidence
        df_class = df_class[df_class['confidence'] > confidence_threshold]

        # Apply NMS
        df_class_sorted = df_class.sort_values(by='confidence', ascending=False).reset_index(drop=True)
        suppressed_indices = set()

        for i in range(len(df_class_sorted)):
            if i in suppressed_indices:
                continue

            for j in range(i+1, len(df_class_sorted)):
                if j in suppressed_indices:
                    continue

                boxA = df_class_sorted.iloc[i][['xmin', 'ymin', 'xmax', 'ymax']]
                boxB = df_class_sorted.iloc[j][['xmin', 'ymin', 'xmax', 'ymax']]

                if compute_iou(boxA, boxB) > iou_threshold:
                    suppressed_indices.add(j)

        # Filter out suppressed detections
        df_nms_class = df_class_sorted.drop(index=list(suppressed_indices)).reset_index(drop=True)

        # Append results for this class to the main DataFrame
        df_nms = pd.concat([df_nms, df_nms_class], ignore_index=True)

    return df_nms

In [ ]:
# import pandas as pd

# def compute_iou(boxA, boxB):
#     # Determine the coordinates of the intersection rectangle
#     xA = max(boxA[0], boxB[0])
#     yA = max(boxA[1], boxB[1])
#     xB = min(boxA[2], boxB[2])
#     yB = min(boxA[3], boxB[3])

#     # Compute the area of intersection
#     interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

#     # Compute the area of both the prediction and ground-truth rectangles
#     boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
#     boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

#     # Compute the intersection over union (IoU)
#     iou = interArea / float(boxAArea + boxBArea - interArea)

#     return iou

# def nms_per_class(df, iou_threshold=0.5):
#     # Initialize an empty DataFrame to store NMS results and IoU values
#     df_nms = pd.DataFrame()

#     # Get unique class IDs
#     class_ids = df['class'].unique()

#     for class_id in class_ids:
#         # Filter detections by class
#         df_class = df[df['class'] == class_id].copy()

#         # Apply NMS
#         df_class_sorted = df_class.sort_values(by='confidence', ascending=False).reset_index(drop=True)
#         retained_indices = []
#         iou_values = []

#         # Collect IoU values and determine retained detections
#         for i in range(len(df_class_sorted)):
#             if i in retained_indices:
#                 continue

#             retained_indices.append(i)
#             iou_values.append(1.0)  # Default IoU value, replace with actual value

#             for j in range(i + 1, len(df_class_sorted)):
#                 if j in retained_indices:
#                     continue

#                 boxA = df_class_sorted.iloc[i][['xmin', 'ymin', 'xmax', 'ymax']]
#                 boxB = df_class_sorted.iloc[j][['xmin', 'ymin', 'xmax', 'ymax']]
#                 iou_value = compute_iou(boxA, boxB)

#                 if iou_value > iou_threshold:
#                     retained_indices.append(j)
#                     iou_values.append(iou_value)

#         # Filter out suppressed detections based on retained indices
#         df_nms_class = df_class_sorted.iloc[retained_indices].reset_index(drop=True)

#         # Add IoU values as a new column to the DataFrame
#         df_nms_class['iou'] = iou_values

#         # Append results for this class to the main DataFrame
#         df_nms = pd.concat([df_nms, df_nms_class], ignore_index=True)

#     return df_nms


#### def map box ที่ detectกับ SHAP value

##### pos img

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from PIL import Image
import numpy as np

def plot_bboxes_on_image_pos(image_path, df, grayscale_image, output_path):
    selected_bboxes = []  #สร้าง list ว่าง มาเก็บกล่องที่มีค่า SHAP เกิน IoU ที่ 0.1
    # value_than_IoU = []

    # Load the original image
    img = Image.open(image_path) #  เปิดภาพจาก path
    img_array = np.array(img)

    # Get the image dimensions
    image_width, image_height = img.size

    # Create figure and axes
    fig, ax = plt.subplots()  #plot ภาพ + กล่อง

    # Display the original image
    ax.imshow(img)

    # Overlay the grayscale image with transparency // plot จากสีเทา ให้เป้นน สีแดง ทับลงไป
    ax.imshow(grayscale_image, cmap='Reds', alpha=0.6, extent=[0, image_width, image_height, 0])

    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():  #df ที่ตัวแบบทนายมาทั้งหมด  จะทำการ loop เก็บค่าตห.จากตาราง
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        confidence, class_label,class_name = row['confidence'], row['class'],row['name']     #,IoU,row['iou']

        # Convert coordinates to absolute values // เปลี่ยนจากตห.ของกล่อง ให้ตามขนาดภาพที่ใส่เข้าไป (ม้้ง)
        abs_xmin = xmin * image_width
        abs_ymin = ymin * image_height
        abs_width = (xmax - xmin) * image_width
        abs_height = (ymax - ymin) * image_height

        # Map bounding box to the grayscale image //คำนวณบริเวณที่สินใจ
        # Calculate the region of interest in the grayscale image
        roi_xmin = int(xmin * grayscale_image.shape[1]) # แปลงพิกัดกรอบสีเหลี่ยมให้เข้ากับขนาดรูปภาพ โดย *พิกัด กับขนาดของ grayscale_image
        roi_ymin = int(ymin * grayscale_image.shape[0])
        roi_xmax = int(xmax * grayscale_image.shape[1])
        roi_ymax = int(ymax * grayscale_image.shape[0])

        # Extract the region of interest from the grayscale image
        roi = grayscale_image[roi_ymin:roi_ymax, roi_xmin:roi_xmax] # เลือก roi บริเวณที่สนใจ  จากgrayscale_image โดยตัด roi ออกมาเป็น nparray

        # Calculate the percentage of nonzero pixels
        nonzero_percentage = np.count_nonzero(roi) / (roi.shape[0] * roi.shape[1]) #Calculate the percentage ของ pixels ที่ไม่เป็น 0 แล้วหารด้วย pixels ทั้งหมด

        # If the percentage of nonzero pixels exceeds 10%, collect the bounding box
        if nonzero_percentage > 0.1:
            # Add bounding box information to the selected_bboxes list
            selected_bboxes.append({'xmin': abs_xmin, 'ymin': abs_ymin,'xmax': abs_xmin + abs_width, 'ymax': abs_ymin + abs_height,
                                    'confidence': confidence, 'class': class_label,'name':class_name}) #, 'iou':IoU
            # value_than_IoU.append(nonzero_percentage)

            # Create a rectangle patch //สร้างกล่อง
            rect = patches.Rectangle(
                (abs_xmin, abs_ymin), #abs แปลงเป็นพิกัดในหน่วย  pixel ให้ตรงกับขนาดภาพจริง
                abs_width,
                abs_height,
                linewidth=2,
                edgecolor='r', # เปลี่ยนสีกล่อง
                facecolor='none'  # Set facecolor to 'none' for an unfilled rectangle
            )

            # Add the rectangle to the axes
            ax.add_patch(rect)
            # Add confidence and class label as text
            text = f'Class: {class_label}' # ข้อความที่จะ Plot บนกล่อง
            #\nConfidence: {confidence:.2f}'
            plt.text(abs_xmin, abs_ymin - 10, text, color='black', fontsize=8, bbox=dict(facecolor='white', alpha=0.5)) # ตห + สีของตัวอักษร ขนาด front
    # Save the figure
    plt.savefig(output_path, bbox_inches='tight')

    # Show the plot
    plt.show()

    return selected_bboxes#, value_than_IoU

##### neg img

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from PIL import Image
import numpy as np

def plot_bboxes_on_image_neg(image_path, df, grayscale_image, output_path):
    selected_bboxes = []
    # value_than_IoU = []

    # Load the original image
    img = Image.open(image_path)
    img_array = np.array(img)

    # Get the image dimensions
    image_width, image_height = img.size

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the original image
    ax.imshow(img)

    # Overlay the grayscale image with transparency
    ax.imshow(grayscale_image, cmap='Blues', alpha=0.6, extent=[0, image_width, image_height, 0])

    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        confidence, class_label,class_name = row['confidence'], row['class'],row['name'] #,IoU,row['iou']

        # Convert coordinates to absolute values
        abs_xmin = xmin * image_width
        abs_ymin = ymin * image_height
        abs_width = (xmax - xmin) * image_width
        abs_height = (ymax - ymin) * image_height

        # Map bounding box to the grayscale image
        # Calculate the region of interest in the grayscale image
        roi_xmin = int(xmin * grayscale_image.shape[1])
        roi_ymin = int(ymin * grayscale_image.shape[0])
        roi_xmax = int(xmax * grayscale_image.shape[1])
        roi_ymax = int(ymax * grayscale_image.shape[0])

        # Extract the region of interest from the grayscale image
        roi = grayscale_image[roi_ymin:roi_ymax, roi_xmin:roi_xmax]

        # Calculate the percentage of nonzero pixels
        nonzero_percentage = np.count_nonzero(roi) / (roi.shape[0] * roi.shape[1])

        # If the percentage of nonzero pixels exceeds 10%, collect the bounding box#เลือกกล่องที่มีค่า shap ในกล่องมากกว่า 10% ของทั้งหมด
        if nonzero_percentage > 0.1:
            # Add bounding box information to the selected_bboxes list
            selected_bboxes.append({'xmin': abs_xmin, 'ymin': abs_ymin,'xmax': abs_xmin + abs_width, 'ymax': abs_ymin + abs_height,
                                    'confidence': confidence, 'class': class_label,'name':class_name}) #, 'iou':IoU
            # value_than_IoU.append(nonzero_percentage)

            # Create a rectangle patch
            rect = patches.Rectangle(
                (abs_xmin, abs_ymin),
                abs_width,
                abs_height,
                linewidth=2,
                edgecolor='b',
                facecolor='none'  # Set facecolor to 'none' for an unfilled rectangle
            )

            # Add the rectangle to the axes
            ax.add_patch(rect)
            # Add confidence and class label as text
            text = f'Class: {class_label}'
            #\nConfidence: {confidence:.2f}'
            plt.text(abs_xmin, abs_ymin - 10, text, color='black', fontsize=8, bbox=dict(facecolor='white', alpha=0.5))
   # Save the figure
    plt.savefig(output_path, bbox_inches='tight')

    # Show the plot
    plt.show()

    return selected_bboxes#, value_than_IoU

# ประมาณค่าอายุถูก (291)

In [ ]:
test_age = pd.read_csv (r'/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_age_all.csv')
test_age = test_age.iloc[:,1:]
print(test_age.shape)
test_age

In [ ]:
print(test_age.shape)
test_age.head(5)

In [ ]:
list_test_age = test_age['Path_Name'].tolist()
list_test_age

In [ ]:
file_paths_test_age = pd.Series(list_test_age)

In [ ]:
# Iterate over each file path in the Series
print(len(file_paths_test_age))
for index, file_path in file_paths_test_age.items():
    print(f"Processing image {index}: {file_path}")


In [ ]:
# # Iterate over each file path in the Series
# dataframe_neg = []
# dataframe_pos = []
# list_neg = []
# list_pos = []

# for index, file_path in file_paths_test_age.items():
#     filename = file_path.split("/")[-1]
#     print(f"Processing image {index}: {file_path}")

#     try:
#         # Load the image
#         image = Image.open(file_path)

#         # Perform your existing image processing steps
#         results = yolo_model(image)  # Assuming yolo_model is defined
#         df = results.pandas().xyxyn[0]  # Assuming results has a pandas method
#         df_filtered_nms = nms_per_class(df, iou_threshold=0.5, confidence_threshold=0.6)  # Assuming nms_per_class is defined
#         shap_values = explainer_layer1.shap_values(background_test_array[[0]])  # Assuming explainer_layer1 and background_test_array are defined

#         # Processing for SHAP values
#         data = [np.array(shap_values)]
#         image_array = data[0]
#         positive = np.where(image_array >= 0, image_array, 0)
#         negative = np.where(image_array < 0, image_array, 0)
#         negative_aps = np.abs(negative)

#         flattened_array_pos = positive.flatten()
#         flattened_array_neg = negative_aps.flatten()

#         normalized_array_pos = (flattened_array_pos - np.min(flattened_array_pos)) / (np.max(flattened_array_pos) - np.min(flattened_array_pos))
#         normalized_array_neg = (flattened_array_neg - np.min(flattened_array_neg)) / (np.max(flattened_array_neg) - np.min(flattened_array_neg))

#         normalized_positive = normalized_array_pos.reshape(positive.shape)
#         normalized_neg = normalized_array_neg.reshape(negative_aps.shape)

#         grayscale_image_pos = normalized_positive / 3.0
#         grayscale_image_neg = normalized_neg / 3.0

#         grayscale_image_positive = np.mean(grayscale_image_pos, axis=4)
#         grayscale_image_negative = np.mean(grayscale_image_neg, axis=4)

#         grayscale_image_positive = grayscale_image_positive.squeeze()
#         grayscale_image_negative = grayscale_image_negative.squeeze()

#         percentile_95_pos = np.percentile(grayscale_image_positive, 95)
#         percentile_95_neg = np.percentile(grayscale_image_negative, 95)

#         grayscale_pos_thresholded = grayscale_image_positive
#         grayscale_neg_thresholded = grayscale_image_negative

#         grayscale_pos_thresholded[grayscale_pos_thresholded < percentile_95_pos] = 0
#         grayscale_neg_thresholded[grayscale_neg_thresholded < percentile_95_neg] = 0
#         # Example: Display or save processed results
#         %matplotlib inline
#         shap.image_plot(shap_values, background_test_array[[0]])

#         # Assuming plot_bboxes_on_image_pos and plot_bboxes_on_image_neg are defined
#         selected_bboxes_pos = plot_bboxes_on_image_pos(file_path, df_filtered_nms, grayscale_pos_thresholded, f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images/pos_image_{filename}")

#         # # Save the figure
#         # image_path = f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG-SHAPer/Images/pos_image_{filename}.png"
#         # plt.savefig(image_path, bbox_inches='tight')

#         dataframe_pos.append(pd.DataFrame(selected_bboxes_pos)) # เก็บค่า ทำเป็น dataframe ก่อน
#         list_pos.append(selected_bboxes_pos) # เก็บค่า list
#         # print(type(selected_bboxes_pos))

#         selected_bboxes_neg = plot_bboxes_on_image_neg(file_path, df_filtered_nms, grayscale_neg_thresholded, f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images/neg_image_{filename}")
#         dataframe_neg.append(pd.DataFrame(selected_bboxes_neg))
#         list_neg.append(selected_bboxes_neg)

#     except Exception as e:
#         print(f"Error processing image {index}: {file_path}")
#         print(e)


In [ ]:
# import os # Full
# os.makedirs("/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images_age")
# os.makedirs("/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images_gender")

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np

# Define your functions nms_per_class, plot_bboxes_on_image_pos, and plot_bboxes_on_image_neg here
# Assuming nms_per_class, plot_bboxes_on_image_pos, and plot_bboxes_on_image_neg are defined

# Initialize lists to keep track of data
dataframe_neg = []
dataframe_pos = []
list_neg = []
list_pos = []

# Iterate over each file path in the Series
for index, file_path in file_paths_test_age.items():
    filename = file_path.split("/")[-1]
    print(f"Processing image {index}: {file_path}")

    try:
        # Load the image
        image = Image.open(file_path)

        # Perform your existing image processing steps
        results = yolo_model(image)  # Assuming yolo_model is defined
        df = results.pandas().xyxyn[0]  # Assuming results has a pandas method
        df_filtered_nms = nms_per_class(df, iou_threshold=0.5, confidence_threshold=0.6)  # Assuming nms_per_class is defined
        shap_values = explainer_layer1.shap_values(background_test_array[[index]])  # Assuming explainer_layer1 and background_test_array are defined

        # Processing for SHAP values
        data = [np.array(shap_values)]
        image_array = data[0]
        positive = np.where(image_array >= 0, image_array, 0)
        negative = np.where(image_array < 0, image_array, 0)
        negative_aps = np.abs(negative)

        flattened_array_pos = positive.flatten()
        flattened_array_neg = negative_aps.flatten()

        normalized_array_pos = (flattened_array_pos - np.min(flattened_array_pos)) / (np.max(flattened_array_pos) - np.min(flattened_array_pos))
        normalized_array_neg = (flattened_array_neg - np.min(flattened_array_neg)) / (np.max(flattened_array_neg) - np.min(flattened_array_neg))

        normalized_positive = normalized_array_pos.reshape(positive.shape)
        normalized_neg = normalized_array_neg.reshape(negative_aps.shape)

        grayscale_image_pos = normalized_positive / 3.0
        grayscale_image_neg = normalized_neg / 3.0

        grayscale_image_positive = np.mean(grayscale_image_pos, axis=4)
        grayscale_image_negative = np.mean(grayscale_image_neg, axis=4)

        grayscale_image_positive = grayscale_image_positive.squeeze()
        grayscale_image_negative = grayscale_image_negative.squeeze()

        percentile_95_pos = np.percentile(grayscale_image_positive, 95)
        percentile_95_neg = np.percentile(grayscale_image_negative, 95)

        grayscale_pos_thresholded = grayscale_image_positive
        grayscale_neg_thresholded = grayscale_image_negative

        grayscale_pos_thresholded[grayscale_pos_thresholded < percentile_95_pos] = 0
        grayscale_neg_thresholded[grayscale_neg_thresholded < percentile_95_neg] = 0

        # Example: Display or save processed results
        %matplotlib inline
        shap.image_plot(shap_values, background_test_array[[index]])

        # Assuming plot_bboxes_on_image_pos and plot_bboxes_on_image_neg are defined
        pos_image_path = f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images_age/pos_image_{filename}"
        selected_bboxes_pos = plot_bboxes_on_image_pos(file_path, df_filtered_nms, grayscale_pos_thresholded, pos_image_path)

        # Add the filename and path to the DataFrame for positive bboxes
        df_pos = pd.DataFrame(selected_bboxes_pos)
        df_pos['filename'] = filename
        df_pos['path_name_ori'] = file_path
        df_pos['path_pos_img'] = pos_image_path
        dataframe_pos.append(df_pos)  # Add DataFrame to the list
        list_pos.append(selected_bboxes_pos)  # Add list to the list

        neg_image_path = f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images_age/neg_image_{filename}"
        selected_bboxes_neg = plot_bboxes_on_image_neg(file_path, df_filtered_nms, grayscale_neg_thresholded, neg_image_path)

        # Add the filename and path to the DataFrame for negative bboxes
        df_neg = pd.DataFrame(selected_bboxes_neg)
        df_neg['filename'] = filename
        df_neg['path_name_ori'] = file_path
        df_neg['path_neg_img'] = neg_image_path
        dataframe_neg.append(df_neg)  # Add DataFrame to the list
        list_neg.append(selected_bboxes_neg)  # Add list to the list

    except Exception as e:
        print(f"Error processing image {index}: {file_path}")
        print(e)

# Concatenate all the DataFrames into a single DataFrame
final_df_pos = pd.concat(dataframe_pos, ignore_index=True)
final_df_neg = pd.concat(dataframe_neg, ignore_index=True)

# Save the final DataFrames to CSV files if needed
final_df_pos.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Result/test_all_age_positive_bboxes.csv', index=False)
final_df_neg.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Result/test_all_age_negative_bboxes.csv', index=False)


# ทำนายเพศถูก prob > 90% (753)

In [ ]:
test_gender = pd.read_csv (r'/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Data/test_gender_Prob_morethan_90.csv')
test_gender = test_gender.iloc[:,1:]
print(test_gender.shape)
test_gender

(753, 14)


,Age(year),Gender,Folder_type,Folder_name,Path_Name,...,Age_predict_int,Gender_predict,Gender_predict_str,Gender_predict_int,prob
0,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,7,5.141945e-03,Female,0.0,0.994858
1,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,7,4.618836e-02,Female,0.0,0.953812
2,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,8,6.406498e-03,Female,0.0,0.993594
3,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,7,2.021052e-05,Female,0.0,0.999980
4,7,F,Normal,Both,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,7,2.091009e-02,Female,0.0,0.979090
...,...,...,...,...,...,...,...,...,...,...,...
748,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,19,9.307945e-06,Female,0.0,0.999991
749,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,22,1.800820e-05,Female,0.0,0.999982
750,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,20,1.212668e-03,Female,0.0,0.998787
751,25,F,Abnormal,Rt,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,...,21,3.044503e-09,Female,0.0,1.000000


In [ ]:
list_test_gender = test_gender['Path_Name'].tolist()
list_test_gender

['/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V69.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV69.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V70.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV70.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V74.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV74.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V75.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V76.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV76.jpg',
 '/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F

In [ ]:
file_paths_test_gender = pd.Series(list_test_gender)

In [ ]:
# Iterate over each file path in the Series
print(len(file_paths_test_gender))
for index, file_path in file_paths_test_gender.items():
    print(f"Processing image {index}: {file_path}")

753
Processing image 0: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V69.jpg
Processing image 1: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV69.jpg
Processing image 2: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V70.jpg
Processing image 3: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV70.jpg
Processing image 4: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V74.jpg
Processing image 5: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/FlipV74.jpg
Processing image 6: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V75.jpg
Processing image 7: /content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Data_Cut_images/Test/Normal/7Y/F/Both/V76.jpg
Processing image 8: /content/gdrive/MyDrive/Tooth_Shap_G

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np

# Define your functions nms_per_class, plot_bboxes_on_image_pos, and plot_bboxes_on_image_neg here
# Assuming nms_per_class, plot_bboxes_on_image_pos, and plot_bboxes_on_image_neg are defined

# Initialize lists to keep track of data
dataframe_neg = []
dataframe_pos = []
list_neg = []
list_pos = []

# Iterate over each file path in the Series
for index, file_path in file_paths_test_gender.items():
    filename = file_path.split("/")[-1]
    print(f"Processing image {index}: {file_path}")

    try:
        # Load the image
        image = Image.open(file_path)

        # Perform your existing image processing steps
        results = yolo_model(image)  # Assuming yolo_model is defined
        df = results.pandas().xyxyn[0]  # Assuming results has a pandas method
        df_filtered_nms = nms_per_class(df, iou_threshold=0.5, confidence_threshold=0.6)  # Assuming nms_per_class is defined
        shap_values = explainer_layer2.shap_values(background_test_array[[index]])  # Assuming explainer_layer1 and background_test_array are defined

        # Processing for SHAP values
        data = [np.array(shap_values)]
        image_array = data[0]
        positive = np.where(image_array >= 0, image_array, 0)
        negative = np.where(image_array < 0, image_array, 0)
        negative_aps = np.abs(negative)

        flattened_array_pos = positive.flatten()
        flattened_array_neg = negative_aps.flatten()

        normalized_array_pos = (flattened_array_pos - np.min(flattened_array_pos)) / (np.max(flattened_array_pos) - np.min(flattened_array_pos))
        normalized_array_neg = (flattened_array_neg - np.min(flattened_array_neg)) / (np.max(flattened_array_neg) - np.min(flattened_array_neg))

        normalized_positive = normalized_array_pos.reshape(positive.shape)
        normalized_neg = normalized_array_neg.reshape(negative_aps.shape)

        grayscale_image_pos = normalized_positive / 3.0
        grayscale_image_neg = normalized_neg / 3.0

        grayscale_image_positive = np.mean(grayscale_image_pos, axis=4)
        grayscale_image_negative = np.mean(grayscale_image_neg, axis=4)

        grayscale_image_positive = grayscale_image_positive.squeeze()
        grayscale_image_negative = grayscale_image_negative.squeeze()

        percentile_95_pos = np.percentile(grayscale_image_positive, 95)
        percentile_95_neg = np.percentile(grayscale_image_negative, 95)

        grayscale_pos_thresholded = grayscale_image_positive
        grayscale_neg_thresholded = grayscale_image_negative

        grayscale_pos_thresholded[grayscale_pos_thresholded < percentile_95_pos] = 0
        grayscale_neg_thresholded[grayscale_neg_thresholded < percentile_95_neg] = 0

        # Example: Display or save processed results
        %matplotlib inline
        shap.image_plot(shap_values, background_test_array[[index]])

        # Assuming plot_bboxes_on_image_pos and plot_bboxes_on_image_neg are defined
        pos_image_path = f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images_gender/pos_image_{filename}"
        selected_bboxes_pos = plot_bboxes_on_image_pos(file_path, df_filtered_nms, grayscale_pos_thresholded, pos_image_path)

        # Add the filename and path to the DataFrame for positive bboxes
        df_pos = pd.DataFrame(selected_bboxes_pos)
        df_pos['filename'] = filename
        df_pos['path_name_ori'] = file_path
        df_pos['path_pos_img'] = pos_image_path
        dataframe_pos.append(df_pos)  # Add DataFrame to the list
        list_pos.append(selected_bboxes_pos)  # Add list to the list

        neg_image_path = f"/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Images_gender/neg_image_{filename}"
        selected_bboxes_neg = plot_bboxes_on_image_neg(file_path, df_filtered_nms, grayscale_neg_thresholded, neg_image_path)

        # Add the filename and path to the DataFrame for negative bboxes
        df_neg = pd.DataFrame(selected_bboxes_neg)
        df_neg['filename'] = filename
        df_neg['path_name_ori'] = file_path
        df_neg['path_neg_img'] = neg_image_path
        dataframe_neg.append(df_neg)  # Add DataFrame to the list
        list_neg.append(selected_bboxes_neg)  # Add list to the list

    except Exception as e:
        print(f"Error processing image {index}: {file_path}")
        print(e)

# Concatenate all the DataFrames into a single DataFrame
final_df_pos = pd.concat(dataframe_pos, ignore_index=True)
final_df_neg = pd.concat(dataframe_neg, ignore_index=True)

# Save the final DataFrames to CSV files if needed
final_df_pos.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Result/test_all_gender_positive_bboxes.csv', index=False)
final_df_neg.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Result/test_all_gender_negative_bboxes.csv', index=False)

In [ ]:
print(len(final_df_neg['filename'].unique()))
final_df_neg['filename'].unique()

749


array(['V69.jpg', 'FlipV69.jpg', 'V70.jpg', 'FlipV70.jpg', 'V74.jpg', 'FlipV74.jpg', 'V75.jpg', 'V76.jpg', 'FlipV76.jpg', 'V78.jpg', 'FlipV78.jpg', 'V79.jpg', 'FlipV79.jpg', 'V81.jpg', 'FlipV81.jpg', 'FlipV84.jpg', 'FlipV85.jpg', 'V68.jpg', 'V67.jpg', 'FlipV67.jpg', 'V88.jpg', 'VV83.jpg', 'VV85.jpg', 'FlipVV85.jpg',
       'FlipVV88.jpg', 'VV89.jpg', 'FlipVV89.jpg', 'FlipVV91.jpg', 'FlipVV98.jpg', 'VV107.jpg', 'FlipVV107.jpg', 'VV108.jpg', 'FlipVV108.jpg', 'P70.jpg', 'FlipP70.jpg', 'FlipP72.jpg', 'FlipP74.jpg', 'FlipP75.jpg', 'P79.jpg', 'FlipP79.jpg', 'P80.jpg', 'FlipP80.jpg', 'P84.jpg', 'FlipP84.jpg', 'FlipP73.jpg', 'P77.jpg',
       'FlipP77.jpg', 'P82.jpg', 'FlipP82.jpg', 'PP68.jpg', 'FlipPP68.jpg', 'PP70.jpg', 'FlipPP70.jpg', 'PP72.jpg', 'FlipPP72.jpg', 'PP75.jpg', 'FlipPP75.jpg', 'PP76.jpg', 'FlipPP76.jpg', 'PP79.jpg', 'FlipPP79.jpg', 'PP80.jpg', 'FlipPP80.jpg', 'PP81.jpg', 'PP83.jpg', 'FlipPP83.jpg', 'FlipPP77.jpg', 'W81.jpg',
       'W85.jpg', 'W87.jpg', 'W89.jpg', 'FlipW89.jpg'

In [ ]:
print(len(final_df_pos['filename'].unique()))
final_df_pos['filename'].unique()

750


array(['V69.jpg', 'FlipV69.jpg', 'V70.jpg', 'FlipV70.jpg', 'V74.jpg', 'FlipV74.jpg', 'V75.jpg', 'V76.jpg', 'FlipV76.jpg', 'V78.jpg', 'FlipV78.jpg', 'V79.jpg', 'FlipV79.jpg', 'V81.jpg', 'FlipV81.jpg', 'FlipV84.jpg', 'FlipV85.jpg', 'V68.jpg', 'V67.jpg', 'FlipV67.jpg', 'V88.jpg', 'VV83.jpg', 'VV85.jpg', 'FlipVV85.jpg',
       'FlipVV88.jpg', 'VV89.jpg', 'FlipVV89.jpg', 'FlipVV91.jpg', 'FlipVV98.jpg', 'VV107.jpg', 'FlipVV107.jpg', 'VV108.jpg', 'FlipVV108.jpg', 'P70.jpg', 'FlipP70.jpg', 'FlipP72.jpg', 'FlipP74.jpg', 'FlipP75.jpg', 'P79.jpg', 'FlipP79.jpg', 'P80.jpg', 'FlipP80.jpg', 'P84.jpg', 'FlipP84.jpg', 'FlipP73.jpg', 'P77.jpg',
       'FlipP77.jpg', 'P82.jpg', 'FlipP82.jpg', 'PP68.jpg', 'FlipPP68.jpg', 'PP70.jpg', 'FlipPP70.jpg', 'PP72.jpg', 'FlipPP72.jpg', 'PP75.jpg', 'FlipPP75.jpg', 'PP76.jpg', 'FlipPP76.jpg', 'PP79.jpg', 'FlipPP79.jpg', 'PP80.jpg', 'FlipPP80.jpg', 'PP81.jpg', 'PP83.jpg', 'FlipPP83.jpg', 'FlipPP77.jpg', 'W81.jpg',
       'W85.jpg', 'W87.jpg', 'W89.jpg', 'FlipW89.jpg'

### final table of positive SHAP values (750 filenames)

In [ ]:
print(len(final_df_pos['filename'].unique()))
final_df_pos['filename'].unique()

750


array(['V69.jpg', 'FlipV69.jpg', 'V70.jpg', 'FlipV70.jpg', 'V74.jpg', 'FlipV74.jpg', 'V75.jpg', 'V76.jpg', 'FlipV76.jpg', 'V78.jpg', 'FlipV78.jpg', 'V79.jpg', 'FlipV79.jpg', 'V81.jpg', 'FlipV81.jpg', 'FlipV84.jpg', 'FlipV85.jpg', 'V68.jpg', 'V67.jpg', 'FlipV67.jpg', 'V88.jpg', 'VV83.jpg', 'VV85.jpg', 'FlipVV85.jpg',
       'FlipVV88.jpg', 'VV89.jpg', 'FlipVV89.jpg', 'FlipVV91.jpg', 'FlipVV98.jpg', 'VV107.jpg', 'FlipVV107.jpg', 'VV108.jpg', 'FlipVV108.jpg', 'P70.jpg', 'FlipP70.jpg', 'FlipP72.jpg', 'FlipP74.jpg', 'FlipP75.jpg', 'P79.jpg', 'FlipP79.jpg', 'P80.jpg', 'FlipP80.jpg', 'P84.jpg', 'FlipP84.jpg', 'FlipP73.jpg', 'P77.jpg',
       'FlipP77.jpg', 'P82.jpg', 'FlipP82.jpg', 'PP68.jpg', 'FlipPP68.jpg', 'PP70.jpg', 'FlipPP70.jpg', 'PP72.jpg', 'FlipPP72.jpg', 'PP75.jpg', 'FlipPP75.jpg', 'PP76.jpg', 'FlipPP76.jpg', 'PP79.jpg', 'FlipPP79.jpg', 'PP80.jpg', 'FlipPP80.jpg', 'PP81.jpg', 'PP83.jpg', 'FlipPP83.jpg', 'FlipPP77.jpg', 'W81.jpg',
       'W85.jpg', 'W87.jpg', 'W89.jpg', 'FlipW89.jpg'

In [ ]:
list_path = list(final_df_pos['path_name_ori']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ

['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'FlipV69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_

In [ ]:
list_path = list(final_df_pos['path_name_ori']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
class_year = list() # สร้างตัวแปร class_list มาเก็บ list ว่าง
class_gender = list() # สร้างตัวแปร class_list มาเก็บ list ว่าง
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ
    class_year.append(res[9]) # append  คำสั่งเพิ่มข้อมูลลงใน List
    class_gender.append(res[10]) # append  คำสั่งเพิ่มข้อมูลลงใน List

final_df_pos['Age(year)'] = class_year # ตั้งชื่อคอลัมน์ว่า Part ในตาราง pathname โดยนำค่าใน class_list มาใส่
final_df_pos['Gender'] = class_gender

['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'FlipV69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_

In [ ]:
final_df_pos

,xmin,ymin,xmax,ymax,confidence,...,filename,path_name_ori,path_pos_img,Age(year),Gender
0,290.441006,318.575717,347.501520,375.691668,0.949560,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
1,442.243241,190.380302,481.506713,277.754403,0.903446,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
2,203.511023,295.167587,268.920413,389.243537,0.901955,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
3,484.279613,233.741133,517.716436,341.186309,0.900649,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
4,463.243440,229.484183,503.849152,327.621257,0.876810,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
...,...,...,...,...,...,...,...,...,...,...,...
4861,154.979572,200.864889,187.619801,281.680926,0.734608,...,J158.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F
4862,264.395053,214.942687,326.121222,303.821259,0.925660,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F
4863,221.766519,199.069218,286.633940,291.172984,0.894681,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F
4864,387.413531,219.669881,435.126467,338.435174,0.886130,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F


In [ ]:
positive_gender = final_df_pos[['ymin', 'xmax', 'ymax', 'confidence', 'class', 'name', 'filename', 'Age(year)', 'Gender', 'path_name_ori', 'path_pos_img']]
positive_gender

,ymin,xmax,ymax,confidence,class,...,filename,Age(year),Gender,path_name_ori,path_pos_img
0,318.575717,347.501520,375.691668,0.949560,13.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
1,190.380302,481.506713,277.754403,0.903446,19.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
2,295.167587,268.920413,389.243537,0.901955,3.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
3,233.741133,517.716436,341.186309,0.900649,17.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4,229.484183,503.849152,327.621257,0.876810,1.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
...,...,...,...,...,...,...,...,...,...,...,...
4861,200.864889,187.619801,281.680926,0.734608,16.0,...,J158.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4862,214.942687,326.121222,303.821259,0.925660,8.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4863,199.069218,286.633940,291.172984,0.894681,15.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4864,219.669881,435.126467,338.435174,0.886130,14.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...


In [ ]:
positive_gender.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Result/positive_gender_95_final.csv')

### final table of negative SHAP values (749 filenames)

In [ ]:
print(len(final_df_neg['filename'].unique()))
final_df_neg['filename'].unique()

749


array(['V69.jpg', 'FlipV69.jpg', 'V70.jpg', 'FlipV70.jpg', 'V74.jpg', 'FlipV74.jpg', 'V75.jpg', 'V76.jpg', 'FlipV76.jpg', 'V78.jpg', 'FlipV78.jpg', 'V79.jpg', 'FlipV79.jpg', 'V81.jpg', 'FlipV81.jpg', 'FlipV84.jpg', 'FlipV85.jpg', 'V68.jpg', 'V67.jpg', 'FlipV67.jpg', 'V88.jpg', 'VV83.jpg', 'VV85.jpg', 'FlipVV85.jpg',
       'FlipVV88.jpg', 'VV89.jpg', 'FlipVV89.jpg', 'FlipVV91.jpg', 'FlipVV98.jpg', 'VV107.jpg', 'FlipVV107.jpg', 'VV108.jpg', 'FlipVV108.jpg', 'P70.jpg', 'FlipP70.jpg', 'FlipP72.jpg', 'FlipP74.jpg', 'FlipP75.jpg', 'P79.jpg', 'FlipP79.jpg', 'P80.jpg', 'FlipP80.jpg', 'P84.jpg', 'FlipP84.jpg', 'FlipP73.jpg', 'P77.jpg',
       'FlipP77.jpg', 'P82.jpg', 'FlipP82.jpg', 'PP68.jpg', 'FlipPP68.jpg', 'PP70.jpg', 'FlipPP70.jpg', 'PP72.jpg', 'FlipPP72.jpg', 'PP75.jpg', 'FlipPP75.jpg', 'PP76.jpg', 'FlipPP76.jpg', 'PP79.jpg', 'FlipPP79.jpg', 'PP80.jpg', 'FlipPP80.jpg', 'PP81.jpg', 'PP83.jpg', 'FlipPP83.jpg', 'FlipPP77.jpg', 'W81.jpg',
       'W85.jpg', 'W87.jpg', 'W89.jpg', 'FlipW89.jpg'

In [ ]:
list_path = list(final_df_neg['path_name_ori']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ

['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'FlipV69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'FlipV69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'To

In [ ]:
list_path = list(final_df_neg['path_name_ori']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
class_year = list() # สร้างตัวแปร class_list มาเก็บ list ว่าง
class_gender = list() # สร้างตัวแปร class_list มาเก็บ list ว่าง
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ
    class_year.append(res[9]) # append  คำสั่งเพิ่มข้อมูลลงใน List
    class_gender.append(res[10]) # append  คำสั่งเพิ่มข้อมูลลงใน List

final_df_neg['Age(year)'] = class_year # ตั้งชื่อคอลัมน์ว่า Part ในตาราง pathname โดยนำค่าใน class_list มาใส่
final_df_neg['Gender'] = class_gender

['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'V69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'FlipV69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Tooth_Shap_GPT', 'Deep_tooth', 'Data_Cut_images', 'Test', 'Normal', '7Y', 'F', 'Both', 'FlipV69.jpg']
['', 'content', 'gdrive', 'MyDrive', 'To

In [ ]:
final_df_neg

,xmin,ymin,xmax,ymax,confidence,...,filename,path_name_ori,path_neg_img,Age(year),Gender
0,290.441006,318.575717,347.501520,375.691668,0.949560,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
1,203.511023,295.167587,268.920413,389.243537,0.901955,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
2,484.279613,233.741133,517.716436,341.186309,0.900649,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
3,463.243440,229.484183,503.849152,327.621257,0.876810,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
4,197.709297,221.778334,218.693814,295.307760,0.808703,...,V69.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,7Y,F
...,...,...,...,...,...,...,...,...,...,...,...
4748,452.864825,224.150105,490.957514,348.404068,0.898716,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F
4749,221.766519,199.069218,286.633940,291.172984,0.894681,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F
4750,387.413531,219.669881,435.126467,338.435174,0.886130,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F
4751,420.126534,205.309653,459.493027,346.118778,0.875917,...,J177.jpg,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,25Y,F


In [ ]:
negative_gender = final_df_neg[['ymin', 'xmax', 'ymax', 'confidence', 'class', 'name', 'filename', 'Age(year)', 'Gender', 'path_name_ori', 'path_neg_img']]
negative_gender

,ymin,xmax,ymax,confidence,class,...,filename,Age(year),Gender,path_name_ori,path_neg_img
0,318.575717,347.501520,375.691668,0.949560,13.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
1,295.167587,268.920413,389.243537,0.901955,3.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
2,233.741133,517.716436,341.186309,0.900649,17.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
3,229.484183,503.849152,327.621257,0.876810,1.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4,221.778334,218.693814,295.307760,0.808703,16.0,...,V69.jpg,7Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
...,...,...,...,...,...,...,...,...,...,...,...
4748,224.150105,490.957514,348.404068,0.898716,1.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4749,199.069218,286.633940,291.172984,0.894681,15.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4750,219.669881,435.126467,338.435174,0.886130,14.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...
4751,205.309653,459.493027,346.118778,0.875917,19.0,...,J177.jpg,25Y,F,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...,/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_to...


In [ ]:
negative_gender.to_csv('/content/gdrive/My Drive/Tooth_Shap_GPT/Deep_tooth/OPG_SHAPer/Result/negative_gender_95_final.csv')